In [161]:
import sys
sys.path.append('..')
from utils.paths import get_file_path

import polars as pl
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [81]:
def estilo(fig):
    fig.update_layout(
        title_font=dict(size=24, family="Arial", color="black"),
        xaxis_tickfont=dict(size=12),
        yaxis_tickfont=dict(size=12),
        plot_bgcolor="white",
        paper_bgcolor="white",
        showlegend=False,
        margin=dict(t=80, l=40, r=20, b=120),
    )
    return fig

In [82]:
df = pl.scan_csv(
    get_file_path('cleaned', 'exportaciones'),
    low_memory=True
).collect()

### Eficiencia por vía - Ratio FLETES/FOBDOL

In [83]:
eficiencia_via = (df.group_by("VIA").agg([
    pl.col("FLETES").sum(),
    pl.col("FOBDOL").sum()
]).with_columns(
    (pl.col("FLETES") / pl.col("FOBDOL")).alias("ratio_flete_fob")
).sort("ratio_flete_fob", descending=True))

In [84]:
eficiencia_via

VIA,FLETES,FOBDOL,ratio_flete_fob
str,f64,f64,f64
"""Férreo""",2.0065e8,5.7498e9,0.034896
"""Aéreo""",2.8673e9,9.6764e10,0.029632
"""Terrestre""",1.4822e9,5.7080e10,0.025967
"""Vías navegables interiores""",3.3085e7,1.6036e9,0.020633
"""Instalaciones de transporte fi…",1.7627e8,1.7310e10,0.010183
"""Correo""",1.4805e7,1.5071e9,0.009824
"""Marítimo""",4.8281e9,6.2018e11,0.007785
"""Otro modo de transporte""",1.8423e8,4.7988e10,0.003839
"""Multimodal""",767359.06,3.8181e8,0.00201


In [100]:
fig = px.bar(eficiencia_via, x="VIA", y="ratio_flete_fob",
             title="Eficiencia por vía de transporte (Ratio Flete/FOB)",
             labels={
                 "VIA": "Vías",
                 "ratio_flete_fob": "Ratio flete/FOB"
             })
fig =  estilo(fig)
fig.show()

### Costo de fletes por destino

In [86]:
costo_destino = (df.group_by("PAIS").agg([
    pl.col("FLETES").sum(),
    pl.col("FOBDOL").sum()
]).with_columns(
    (pl.col("FLETES") / pl.col("FOBDOL")).alias("avg_flete_fob")
).sort("avg_flete_fob", descending=True))

In [87]:
costo_destino

PAIS,FLETES,FOBDOL,avg_flete_fob
str,f64,f64,f64
"""Territorio Británico del Océan…",51212.96,175641.1,0.291577
"""Ruanda""",266207.05,1.2992e6,0.204896
"""Burundi""",66436.74,344315.32,0.192953
"""Zimbabue""",1.6504e6,1.0114e7,0.163185
"""Bielorrusia""",3.3486e6,2.0960e7,0.159764
…,…,…,…
"""ZFPE Agroindust""",0.0,123849.26,0.0
"""Vanuatu""",0.0,275078.52,0.0
"""ZFP Tayrona S.A""",0.0,74670.08,0.0


In [88]:
fig = px.bar(costo_destino.head(20), x="PAIS", y="avg_flete_fob",
             title="Costo de fletes por destino (Top 20)",
             labels={
                 "PAIS": "País",
                 "avg_flete_fob": "Ratio flete/FOB"
             })
fig =  estilo(fig)
fig.show()

In [95]:
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Fletes totales por destino', 'Ratio Flete/FOB por destino')
)

fig.add_trace(
    go.Bar(x=costo_destino.head(15)["PAIS"], 
           y=costo_destino.head(15)["FLETES"],
           name="Fletes totales"),
    row=1, col=1
)
fig.update_yaxes(
    type="log",
    range=[3, 9],
    row=1, col=1
)

fig.add_trace(
    go.Bar(x=costo_destino.head(15)["PAIS"], 
           y=costo_destino.head(15)["avg_flete_fob"],
           name="Ratio flete/FOB"),
    row=1, col=2
)
fig = estilo(fig)
fig.update_layout(title_text="Análisis de costos de fletes por destino")

### Participación por vía

In [90]:
total_fob_via = df.select(pl.col("FOBDOL").sum()).item()
participacion_via = (df.group_by("VIA").agg(pl.col("FOBDOL").sum())
                     .with_columns(
                         (pl.col("FOBDOL") / total_fob_via * 100).alias("participacion_pct")
                     ).sort("participacion_pct", descending=True))

In [91]:
total_fob_via

848567007773.75

In [92]:
fig = px.pie(participacion_via, values="participacion_pct", names="VIA",
             title="Participación por Vía de Transporte (%)")
fig.show()

### Corredores principales

##### ¿Qué es un corredor de exportación?

Un corredor de exportación es una ruta comercial específica que conecta:

+ Origen: Departamento donde se produce/origina la mercancía
+ Destino: País al que se exporta
+ Medio: Vía de transporte utilizada (marítima, aérea, terrestre)

Es como una "autopista comercial" que muestra los flujos más importantes de productos desde regiones específicas hacia mercados internacionales.

In [93]:
corredores = (df.group_by(["DEPTO", "PAIS", "VIA"]).agg([
    pl.col("FOBDOL").sum().alias("total_fob"),
    pl.col("FLETES").sum().alias("total_fletes")
]).with_columns(
    (pl.col("total_fletes") / pl.col("total_fob")).alias("ratio_flete_fob")
).sort("total_fob", descending=True).head(15))

In [94]:
corredores_con_label = corredores.with_columns(
    (pl.col("DEPTO") + " → " + pl.col("PAIS") + " (" + pl.col("VIA") + ")").alias("corredor_completo")
)

corredores_con_label = corredores.with_columns([
    pl.col("DEPTO").alias("origen"),
    pl.col("PAIS").alias("destino"),
    pl.col("VIA").alias("via_transporte"),
    (pl.col("DEPTO") + " → " + pl.col("PAIS") + " (" + pl.col("VIA") + ")").alias("ruta_completa")
])

fig = px.bar(corredores_con_label, 
             x="total_fob", 
             y="ruta_completa",
             title="Top 15 Corredores de exportación (Origen → Destino)",
             labels={
                 "total_fob": "Valor FOB (USD)",
                 "ruta_completa": "Ruta de exportación"
             })
fig = estilo(fig)
fig.show()

### Productos más transportados por vía

In [ ]:
productos_via = (df.group_by(["POSARA", "VIA"]).agg([
    pl.col("FOBDOL").sum().alias("total_fob"),
    pl.col("FLETES").sum().alias("total_fletes"),
    pl.col("FOBDOL").count().alias("num_operaciones")
]).with_columns(
    (pl.col("total_fletes") / pl.col("total_fob")).alias("ratio_flete_fob")
).sort("total_fob", descending=True))

In [132]:
top_productos_general = (df.group_by("POSARA").agg([
    pl.col("FOBDOL").sum().alias("total_fob")
]).sort("total_fob", descending=True).head(15))

productos = {
    2709000000: "Petróleo crudo",
    2701120010: "Carbón bituminoso",
    901119000: "Café verde",
    7108120000: "Oro no monetario",
    2710192200: "Aceites preparados",
    7202600000: "Chatarra hierro/acero",
    2704001000: "Coque de carbón",
    2710192100: "Kerosene (jet fuel)",
    803901100: "Banano fresco",
    603199090: "Flores cortadas",
    603110000: "Crisantemos",
    1701999000: "Azúcar refinado",
    3902100000: "Polietileno HDPE",
    3904102000: "Copoliéster acrílico",
    1511100000: "Aceite de soja"
}


top_productos_general = top_productos_general.to_pandas()
top_productos = top_productos_general.copy()
top_productos_general['POSARA'] = top_productos_general['POSARA'].map(productos)
top_productos_general = top_productos_general.rename(columns={'POSARA': 'PRODUCTOS'})

In [151]:
productos_distribucion = (df.filter(pl.col("POSARA").is_in(top_productos["POSARA"].to_list()))
                         .group_by(["POSARA", "VIA"])
                         .agg(pl.col("FOBDOL").sum())
                         .sort(["POSARA", "FOBDOL"], descending=[False, True]))

productos_distribucion = productos_distribucion.to_pandas()
productos_distribucion['POSARA'] = productos_distribucion['POSARA'].map(productos)
productos_distribucion = productos_distribucion.rename(columns={'POSARA': 'PRODUCTOS'})

In [160]:
fig = px.bar(top_productos_general, x="PRODUCTOS", y="total_fob",
              title="Top 15 productos por valor FOB")
fig = estilo(fig)
fig.show()

In [166]:
productos_distribucion["FOBDOL_LOG"] = productos_distribucion["FOBDOL"].apply(lambda x: np.log10(x + 1))

fig = px.density_heatmap(
    productos_distribucion,
    x="PRODUCTOS",
    y="VIA",
    z="FOBDOL_LOG",
    color_continuous_scale="YlGnBu",
    title="Distribución logarítmica de vías por top productos"
)
fig.update_traces(
    hovertemplate=
    "<b>Producto:</b> %{x}<br>" +
    "<b>Vía:</b> %{y}<br>" +
    "<b>Fobdol:</b> %{z:$,.2f}<extra></extra>"
)

fig.update_layout(
    coloraxis_colorbar=dict(
        title="Fobdol",
        ticks="outside",
        tickmode="auto"
    ),
    title_font=dict(size=24, family="Arial", color="black"),
    xaxis_tickfont=dict(size=12),
    yaxis_tickfont=dict(size=12),
    xaxis_title="Producto",
    yaxis_title="Vía de exportación",
    xaxis_tickangle=45
)

fig.show()